In [2]:
import cv2
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.pyplot as plt
import math
from marswinds.utils import radian_to_degree, degree_to_radian
from tensorflow.keras.models import load_model
import joblib
from os import listdir
from os.path import isfile, join

In [3]:
dune_test_path = '../raw_data/images/testing/dunes'
no_dune_test_path = '../raw_data/images/testing/no_dunes'
base_folder='../raw_data/trained_models'

regressor = load_model(f'{base_folder}/regressor.h5')
regressor_2 = load_model(f'{base_folder}/final_post_bootcamp.h5')
final_regressor = load_model(f'{base_folder}/final_post_bootcamp.h5')
regressor_dir = load_model(f'{base_folder}/wind_direction.h5')
classifier = load_model(f'{base_folder}/classifier.h5')
scaler = joblib.load(f'{base_folder}/scaler.lib')


/Users/cedric/.pyenv/versions/marswinds/lib/python3.8/site-packages/sklearn/base.py:310: UserWarning: Trying to unpickle estimator MinMaxScaler from version 0.22 when using version 0.24.2. This might lead to breaking code or invalid results. Use at your own risk.
  warnings.warn(


# Testing the regressor

In [ ]:
test_files = [f for f in listdir(dune_test_path) if len(f)>30]
test_files

In [ ]:
def get_label(image_name):
    parts = image_name[:-4].split('_')
    sin = float(parts[-3])
    cos = float(parts[-2])
    angle = radian_to_degree(math.atan2(sin, cos))
    wind_strength = float(parts[-1])
    
    return(wind_strength, angle)

In [ ]:
labels = [get_label(f) for f in test_files]
labels

In [ ]:
def prediction_from_models(image_name):
        image_path = f'{dune_test_path}/{image_name}'
        try:
            image = cv2.imread(image_path)
            grey_image = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY) / 255.0
            expanded = np.expand_dims(np.expand_dims(grey_image,axis=0), axis=3)
            predicted_regressed = final_regressor.predict(expanded)[0]
            wind_strength = predicted_regressed[0]
            sin = predicted_regressed[2] # Changed to match Aris code
            cos = predicted_regressed[1]
            angle_rad = math.atan2(sin, cos)
        except Exception as e:
            print(f'Problem with file {image_path}')
            print(e)
            return (np.nan,np.nan)
        
        return (scaler.inverse_transform([[wind_strength]])[0][0], radian_to_degree(angle_rad))

In [ ]:
from tqdm import tqdm

log_name = '../raw_data/logs/testing_regressor_2.csv'
d = {'filename':[],
    'angular_diff':[],
    'speed_diff':[],
    'predicted_angle':[],
    'actual_angle':[]}

df = pd.DataFrame.from_dict(d).to_csv(log_name, index=False)

def get_diff(files):
    
    for file in tqdm(files):
        pred_wind, pred_angle = prediction_from_models(file)
        label_wind, label_angle = get_label(file)
        
        wind_diff = np.abs(label_wind-pred_wind)
        angle_diff = np.abs(label_angle-pred_angle)
        if angle_diff > 180: angle_diff = 360-angle_diff
            
        d = {'filename':[file],
        'angular_diff':[angle_diff**2],
        'speed_diff':[wind_diff**2],
    'predicted_angle':[pred_angle],
    'actual_angle':[label_angle]}

        pd.concat([pd.read_csv(log_name), pd.DataFrame.from_dict(d)]).to_csv(log_name, index=False)
    
    return pd.read_csv(log_name)
        
        

In [ ]:
get_diff(test_files)


In [ ]:
res = pd.read_csv(log_name)

np.sqrt(res.angular_diff.mean())

In [ ]:
bad_res = res[res.angular_diff>100**2]

In [ ]:
bad_res.drop_duplicates()

In [ ]:
np.sqrt(res.angular_diff.mean())

In [ ]:
mrse_wind_model = np.sqrt(res.speed_diff.mean())

In [ ]:
res.actual_angle.hist()

In [ ]:
res.predicted_angle.hist()

In [ ]:
dune_train_path = '../raw_data/images/training/dunes'
train_files = [f for f in listdir(dune_train_path) if len(f)>30]
train_labels_direction = [get_label(f)[1] for f in train_files]
pd.DataFrame(train_labels_direction).hist()

### Performance of wind direction prediction model

In [ ]:
mean_wind_dir = np.mean(train_labels_direction)
mrse_base_wind_dir = np.sqrt(np.mean((np.array(train_labels_direction)-mean_wind_dir)**2))
mrse_model_wind_dir = np.sqrt(res.angular_diff.mean())

print(f'MRSE our model: {mrse_model_wind_dir}')
print(f'MRSE base prediction: {mrse_base_wind_dir}')

### Performance of wind strength prediction model

In [ ]:
wind_directions_training = [get_label(f)[0] for f in train_files]

mean_wind_direction = np.mean(wind_directions_training)

base_model_score = np.sqrt(np.mean((np.array(wind_directions_training)-mean_wind_direction)**2))
base_model_score

In [ ]:
print(f'MRSE our model: {mrse_wind_model}')
print(f'MRSE base prediction: {base_model_score}')

## Testing Sin/Cos model only

In [ ]:
def get_diff_direction(files):
    
    for file in files:
        pred_angle = predict_direction_from_models(file)
        label_wind, label_angle = get_label(file)
        angle_diff = np.abs(label_angle-pred_angle)
        if angle_diff > 180: angle_diff = 360-angle_diff
            
        d = {'filename':[file],
        'angular_diff':[angle_diff**2],
        'predicted_angle':[pred_angle],
        'actual_angle':[label_angle]}

        pd.concat([pd.read_csv(log_name), pd.DataFrame.from_dict(d)]).to_csv(log_name, index=False)
    
    return pd.read_csv(log_name)
        
        

In [ ]:
def predict_direction_from_models(image_name):
        image_path = f'{dune_test_path}/{image_name}'
        try:
            image = cv2.imread(image_path)
            grey_image = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY) / 255.0
            expanded = np.expand_dims(np.expand_dims(grey_image,axis=0), axis=3)
            predicted_regressed = regressor_dir.predict(expanded)[0]
            sin = predicted_regressed[1] # Changed to match Aris code
            cos = predicted_regressed[0]
            angle_rad = math.atan2(sin, cos)
        except:
            print(f'Problem with file {image_path}')
            return (np.nan,np.nan)
        
        return (radian_to_degree(angle_rad))

In [ ]:
log_name = '../raw_data/logs/testing_wind_dir_regressor.csv'

d = {'filename':[],
    'angular_diff':[],
    'predicted_angle':[],
    'actual_angle':[]}

df = pd.DataFrame.from_dict(d).to_csv(log_name, index=False)

dune_test_path = '../raw_data/images/testing/dunes'

test_wind_dir_files = [f for f in listdir(dune_test_path) if len(f)>30]

In [ ]:
df = get_diff_direction(test_wind_dir_files)
df

In [ ]:

mrse_model_wind_dir_sin_cos = np.sqrt(df.angular_diff.mean())

print(f'MRSE our model: {mrse_model_wind_dir_sin_cos}')
print(f'MRSE base prediction: {mrse_base_wind_dir}')

# Testing the Classifier

In [4]:
from tqdm import tqdm

def prediction_from_classifier(image_name):
        image_path = f'{dune_test_path}/{image_name}'
        try:
            image = cv2.imread(image_path)
            grey_image = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY) / 255.0
            expanded = np.expand_dims(np.expand_dims(grey_image,axis=0), axis=3)
            predicted_proba = classifier.predict(expanded)[0][0]
            if predicted_proba > 0.5:
                predicted_class = 'dunes'
            else:
                predicted_class = 'no_dunes'
        except:
            print(f'Problem with file {image_path}')
            return (np.nan,np.nan)
        
        return (predicted_proba, predicted_class)

In [8]:
log_name = '../raw_data/logs/testing_classifier2.csv'
d = {'filename':[],
    'actual_class':[],
    'predicted_class':[],
    'predicted_proba':[]}

df = pd.DataFrame.from_dict(d).to_csv(log_name, index=False)

def get_class(files, label):
    
    for file in tqdm(files):
        pred_proba, pred_class = prediction_from_classifier(file)

            
        d = {'filename':[file],
        'actual_class':[label],
        'predicted_class':[pred_class],
        'predicted_proba':[pred_proba]}

        pd.concat([pd.read_csv(log_name), pd.DataFrame.from_dict(d)]).to_csv(log_name, index=False)
    
    return pd.read_csv(log_name)

In [6]:
dune_test_path = '../raw_data/images/testing/dunes'
no_dune_test_path = '../raw_data/images/testing/no_dunes'

is_dune_files = ('dunes', [f for f in listdir(dune_test_path) if len(f)>30])
no_dune_files = ('no_dunes',[f for f in listdir(no_dune_test_path) if len(f)>30])

In [9]:
log_name = '../raw_data/logs/testing_classifier.csv'

for files in [is_dune_files,no_dune_files]:
    get_class(files[1], files[0])


  0%|          | 3/3313 [00:00<02:23, 23.13it/s]

Problem with file ../raw_data/images/testing/dunes/14.757500267028808_48.25750045776367_01_CW000_0.5697365403175354_-0.24281641840934753_1.2468816041946411.jpg
Problem with file ../raw_data/images/testing/dunes/43.09849853515625_100.77872314453126_010_CW000_-0.4079539477825165_0.7680480480194092_2.133735179901123.jpg
Problem with file ../raw_data/images/testing/dunes/33.33850173950195_64.25090179443359_023_CW000_-0.19145871698856354_-0.3099783658981323_0.959148108959198.jpg
Problem with file ../raw_data/images/testing/dunes/20.466499710083006_16.83580093383789_031_CW000_-0.7050067186355591_0.17107044160366058_0.5570440888404846.jpg
Problem with file ../raw_data/images/testing/dunes/38.5854995727539_96.08677520751954_020_CW000_0.1708187609910965_0.8070891499519348_1.4546841382980347.jpg


  0%|          | 9/3313 [00:00<02:29, 22.09it/s]

Problem with file ../raw_data/images/testing/dunes/20.36649932861328_16.38554916381836_033_CW000_-0.06521864980459213_0.15649478137493134_0.6092240214347839.jpg
Problem with file ../raw_data/images/testing/dunes/33.41849975585937_55.486542510986325_021_CW000_-0.6878598928451538_0.35237565636634827_1.3015036582946777.jpg
Problem with file ../raw_data/images/testing/dunes/41.62049942016601_75.97950134277343_023_CW000_0.19551150500774384_-0.18319888412952423_0.7999233603477478.jpg
Problem with file ../raw_data/images/testing/dunes/43.14850082397461_101.1038330078125_031_CW000_-0.1863543689250946_0.8408895134925842_1.9971835613250732.jpg
Problem with file ../raw_data/images/testing/dunes/22.167500305175782_51.19749984741211_023_CW000_-0.7303146719932556_-0.5044719576835632_1.9662197828292847.jpg


  0%|          | 12/3313 [00:00<02:28, 22.21it/s]

...

Problem with file ../raw_data/images/testing/dunes/26.212500762939452_25.722500228881835_010_CW000_-0.7327685356140137_-0.2275446653366089_1.728513479232788.jpg
Problem with file ../raw_data/images/testing/dunes/20.66650047302246_16.43559913635254_031_CW000_-0.4338754713535309_0.15508918464183807_0.5579233169555664.jpg
Problem with file ../raw_data/images/testing/dunes/28.680500411987303_16.931750106811524_031_CW000_-0.3803493082523346_-0.2937868535518646_1.8581100702285767.jpg
Problem with file ../raw_data/images/testing/dunes/31.178500366210937_56.80563201904297_010_CW000_-0.21088147163391113_0.4786773920059204_0.9718106389045715.jpg


100%|█████████▉| 3312/3313 [02:43<00:00, 15.61it/s]

Problem with file ../raw_data/images/testing/dunes/38.38549880981445_96.16177520751953_023_CW000_0.051238764077425_0.8677064776420593_1.4141662120819092.jpg
Problem with file ../raw_data/images/testing/dunes/31.153500366210938_57.48103485107422_023_CW000_-0.750908613204956_0.586715817451477_1.4864286184310913.jpg
Problem with file ../raw_data/images/testing/dunes/41.39549865722656_77.10491027832032_030_CW000_0.6901392936706543_0.11394710838794708_0.8951123952865601.jpg


100%|██████████| 3313/3313 [02:43<00:00, 20.21it/s]


Problem with file ../raw_data/images/testing/dunes/33.413500213623045_64.20090179443359_031_CW000_-0.20786884427070618_-0.4033299684524536_1.0149190425872803.jpg


In [ ]:
results = pd.read_csv(log_name)

In [ ]:
results

In [ ]:
results.predicted_proba == np.nan